In [1]:
# Importing necessary libraries
import os
import random
import numpy as np
import pandas as pd
from PIL import Image
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

os.curdir = "C:\\Users\\bayra\\Desktop\\Courses\\AIN311\\assignment3"

In [2]:
# Reading provided csv files
train = pd.read_csv("train_data.csv")
test = pd.read_csv("test_data.csv")
train.head()

,filename,text,age,gender,accent
0,sample-084720.png,i had seen all that it would presently bring me,twenties,other,england
1,sample-169346.png,a friend had told the boy about the shop and h...,twenties,male,england
2,sample-027740.png,the boy said nothing,twenties,male,indian
3,sample-035454.png,what is the matter,twenties,male,us
4,sample-134062.png,no baselines or comparison to state of the art...,twenties,female,australia


In [3]:
# Dropping text, gender and accent columns.
train.drop(columns=["text", "gender", "accent"], inplace=True)
test.drop(columns=["text", "gender", "accent"], inplace=True)
train.head()

,filename,age
0,sample-084720.png,twenties
1,sample-169346.png,twenties
2,sample-027740.png,twenties
3,sample-035454.png,twenties
4,sample-134062.png,twenties


In [4]:
# Replacing age column with integers.
replace = {"teens"   : 0,
           "twenties": 1,
           "thirties": 2,
           "fourties": 3,
           "fifties" : 4,
           "sixties" : 5,
}

train.replace(replace, inplace=True)
test.replace(replace, inplace=True)
train.head()

,filename,age
0,sample-084720.png,1
1,sample-169346.png,1
2,sample-027740.png,1
3,sample-035454.png,1
4,sample-134062.png,1


In [5]:
# Function to load images.
def load_images(df, file):
    X = []
    Y = []
    
    for index, row in df.iterrows():
        image_path = os.curdir + "\\" + file + "\\" + row['filename']
        img = Image.open(image_path).convert("L")
        img = img.resize((100, 100))
        img_array = np.array(img) 
        X.append(img_array/255.0)
        Y.append(row['age'])
        del img, img_array
    return np.array(X), np.array(Y)

In [6]:
# Loading images.
trainX, trainY = load_images(train, "train")
testX, testY = load_images(test, "test")

In [7]:
# Converting labels to one hot encoding.
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False)

trainY = one_hot_encoder.fit_transform(np.array(trainY).reshape(-1, 1))
testY = one_hot_encoder.fit_transform(np.array(testY).reshape(-1, 1))
trainY

array([[0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.]])

In [8]:
# Flattening trainX and testX.
trainX_flt = trainX.reshape((9600,10000))
testX_flt = testX.reshape((2400,10000))

In [9]:
# Neural Network class for MLP.
class NeuralNetwork:
    def __init__(self, input_size, hidden_layers, output_size, activation):
        self.input_size = input_size
        self.hidden_layers = hidden_layers
        self.output_size = output_size
        self.activation = activation
        self.weights, self.biases = self.initialize_parameters()

    # To initialize weight randomly at the beginning
    def initialize_parameters(self):
        sizes = [self.input_size] + self.hidden_layers + [self.output_size]
        weights = [np.random.randn(sizes[i], sizes[i+1]) for i in range(len(sizes)-1)]
        biases = [np.zeros((1, sizes[i+1])) for i in range(len(sizes)-1)]
        return weights, biases

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    def activate(self, x):
        activation_functions = {
            'sigmoid': self.sigmoid,
            'softmax': self.softmax,
        }
        return activation_functions[self.activation](x)

    # Function to perform forward propagation
    def forward_propagation(self, x):
        activations = [x]
        for i in range(len(self.hidden_layers)):
            z = np.dot(activations[-1], self.weights[i]) + self.biases[i]
            a = self.activate(z)
            activations.append(a)
        
        output = np.dot(activations[-1], self.weights[-1]) + self.biases[-1]
        output = self.activate(output)
        activations.append(output)
        
        return activations

    # Funtion to calc the sum of the negative log-likelihood of the correct labels.
    def compute_loss(self, y_true, y_pred):
        loss = -np.sum(y_true * np.log(y_pred + 0.00000001)) / y_true.shape[0]
        return loss
    
    # Funtion to perform back propagation
    def backward_propagation(self, x, y_true, activations):
        m = x.shape[0]
        gradients = [activations[-1] - y_true]
        
        # Backpropagation for hidden layers
        for i in range(len(self.hidden_layers), 0, -1):
            dz = np.dot(gradients[0], self.weights[i].T)
            da = dz * (activations[i] > 0) if self.activation == 'relu' else dz
            dw = np.dot(activations[i-1].T, da) / m
            db = np.sum(da, axis=0, keepdims=True) / m
            gradients.insert(0, da)
            self.weights[i-1] -= self.learning_rate * dw
            self.biases[i-1] -= self.learning_rate * db
        
        # Backpropagation for the input layer
        dw = np.dot(x.T, gradients[0]) / m
        db = np.sum(gradients[0], axis=0, keepdims=True) / m
        self.weights[0] -= self.learning_rate * dw
        self.biases[0] -= self.learning_rate * db

    def train(self, x, y, epochs, learning_rate):
        self.learning_rate = learning_rate
        for epoch in range(epochs):
            activations = self.forward_propagation(x)
            loss = self.compute_loss(y, activations[-1])
            self.backward_propagation(x, y, activations)
            
            if (epoch%5) == 0:
                accuracy = self.compute_accuracy(y, activations[-1])
                print(f"Epoch {epoch}, Loss: {loss}, Accuracy: {accuracy}%")

    def compute_accuracy(self, y_true, y_pred):
        predictions = np.argmax(y_pred, axis=1)
        true_labels = np.argmax(y_true, axis=1)
        correct_predictions = np.sum(predictions == true_labels)
        total_examples = y_true.shape[0]
        accuracy = (correct_predictions / total_examples)
        return accuracy

    def predict(self, x):
        activations = self.forward_propagation(x)
        return activations[-1]

In [10]:
# A dict to store the results.
results = {}
results["Model"] = []
results["Input Size"] = []
results["Activation Func"] = []
results["Hidden Layer Size"] = []
results["Learnin Rate"] = []
results["Accuracy"] = []

# MLP with 0 hidden layers

In [11]:
activations = ['sigmoid', 'softmax']
learnings   = [0.005, 0.010, 0.017]
layer = []
for activation in activations:
    for rate in learnings:
        print(f"Experiment with activation: '{activation}' || learning rate: '{rate}'")
        model = NeuralNetwork(10000, layer, 6, activation)
        model.train(trainX_flt, trainY, epochs=20, learning_rate=rate)
        accuracy = model.compute_accuracy(testY, model.predict(testX_flt))
        print(f"Test Accuracy with activation: '{activation}' || learning rate: '{rate}' = {accuracy}\n")
        results["Model"].append("MLP")
        results["Input Size"].append("100x100x1")
        results["Activation Func"].append(activation)
        results["Hidden Layer Size"].append(layer)
        results["Learnin Rate"].append(rate)
        results["Accuracy"].append(accuracy)

Experiment with activation: 'sigmoid' || learning rate: '0.005'
Epoch 0, Loss: 8.136536952259085, Accuracy: 0.16260416666666666%
Epoch 5, Loss: 9.44068591564693, Accuracy: 0.159375%
Epoch 10, Loss: 9.952744342897653, Accuracy: 0.161875%
Epoch 15, Loss: 10.007350231879945, Accuracy: 0.1615625%
Test Accuracy with activation: 'sigmoid' || learning rate: '0.005' = 0.15791666666666668

Experiment with activation: 'sigmoid' || learning rate: '0.01'
Epoch 0, Loss: 10.206851243003316, Accuracy: 0.16677083333333334%
Epoch 5, Loss: 10.664782528113356, Accuracy: 0.164375%
Epoch 10, Loss: 10.525892672994495, Accuracy: 0.15958333333333333%
Epoch 15, Loss: 9.995340469600178, Accuracy: 0.16302083333333334%
Test Accuracy with activation: 'sigmoid' || learning rate: '0.01' = 0.165

Experiment with activation: 'sigmoid' || learning rate: '0.017'
Epoch 0, Loss: 10.310798805818004, Accuracy: 0.16427083333333334%
Epoch 5, Loss: 10.754555141543019, Accuracy: 0.15427083333333333%
Epoch 10, Loss: 10.215916431

# MLP with 1 hidden layer

In [12]:
activations = ['sigmoid', 'softmax']
learnings   = [0.005, 0.01, 0.017]

for layer in [[64],[128]]:
    for activation in activations:
        for rate in learnings:
            print(f"Experiment with activation:'{activation}' || learning rate:'{rate}' || layer:'{layer}'")
            model = NeuralNetwork(10000, layer, 6, activation)
            model.train(trainX_flt, trainY, epochs=20, learning_rate=rate)
            accuracy = model.compute_accuracy(testY, model.predict(testX_flt))
            print(f"Test Accuracy with activation:'{activation}' || learning rate:'{rate}' || layer:'{layer}' = {accuracy}\n")
            results["Model"].append("MLP")
            results["Input Size"].append("100x100x1")
            results["Activation Func"].append(activation)
            results["Hidden Layer Size"].append(layer)
            results["Learnin Rate"].append(rate)
            results["Accuracy"].append(accuracy)

Experiment with activation:'sigmoid' || learning rate:'0.005' || layer:'[64]'
Epoch 0, Loss: 2.6475761880565125, Accuracy: 0.16322916666666668%
Epoch 5, Loss: 2.859680422740019, Accuracy: 0.16260416666666666%
Epoch 10, Loss: 2.8093831974751358, Accuracy: 0.16489583333333332%
Epoch 15, Loss: 2.7947526236246256, Accuracy: 0.16572916666666668%
Test Accuracy with activation:'sigmoid' || learning rate:'0.005' || layer:'[64]' = 0.165

Experiment with activation:'sigmoid' || learning rate:'0.01' || layer:'[64]'
Epoch 0, Loss: 3.4554129946279053, Accuracy: 0.16458333333333333%
Epoch 5, Loss: 3.1609959920964585, Accuracy: 0.165%
Epoch 10, Loss: 2.999537673186368, Accuracy: 0.16875%
Epoch 15, Loss: 3.6118239241075436, Accuracy: 0.16520833333333335%
Test Accuracy with activation:'sigmoid' || learning rate:'0.01' || layer:'[64]' = 0.15958333333333333

Experiment with activation:'sigmoid' || learning rate:'0.017' || layer:'[64]'
Epoch 0, Loss: 5.440965406610985, Accuracy: 0.16770833333333332%
Epoch

# MLP with 2 hidden layers

In [13]:
activations = ['sigmoid', 'softmax']
learnings   = [0.005, 0.01, 0.017]

for layer in [[128,64],[256,128]]:
    for activation in activations:
        for rate in learnings:
            print(f"Experiment with activation:'{activation}' || learning rate:'{rate}' || layer:'{layer}'")
            model = NeuralNetwork(10000, layer, 6, activation)
            model.train(trainX_flt, trainY, epochs=20, learning_rate=rate)
            accuracy = model.compute_accuracy(testY, model.predict(testX_flt))
            print(f"Test Accuracy with activation:'{activation}' || learning rate:'{rate}' || layer:'{layer}' = {accuracy}\n")
            results["Model"].append("MLP")
            results["Input Size"].append("100x100x1")
            results["Activation Func"].append(activation)
            results["Hidden Layer Size"].append(layer)
            results["Learnin Rate"].append(rate)
            results["Accuracy"].append(accuracy)

Experiment with activation:'sigmoid' || learning rate:'0.005' || layer:'[128, 64]'
Epoch 0, Loss: 2.5811813254531444, Accuracy: 0.16979166666666667%
Epoch 5, Loss: 5.989723539627912, Accuracy: 0.16583333333333333%
Epoch 10, Loss: 6.999226453479582, Accuracy: 0.17125%
Epoch 15, Loss: 4.903949771739373, Accuracy: 0.1675%
Test Accuracy with activation:'sigmoid' || learning rate:'0.005' || layer:'[128, 64]' = 0.16625

Experiment with activation:'sigmoid' || learning rate:'0.01' || layer:'[128, 64]'
Epoch 0, Loss: 1.951566218307645, Accuracy: 0.16895833333333332%
Epoch 5, Loss: 3.1656636776875735, Accuracy: 0.16697916666666668%
Epoch 10, Loss: 2.7330048264393807, Accuracy: 0.16666666666666666%
Epoch 15, Loss: 2.3150065299630778, Accuracy: 0.17114583333333333%
Test Accuracy with activation:'sigmoid' || learning rate:'0.01' || layer:'[128, 64]' = 0.18541666666666667

Experiment with activation:'sigmoid' || learning rate:'0.017' || layer:'[128, 64]'
Epoch 0, Loss: 1.2699727874770157, Accuracy:

# CNN with 1 hidden layer

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import SGD

In [15]:
activations = ['relu','sigmoid']
for layer in [64,128]:
    for activation in activations:
            print(f"Experiment with activation:'{activation}' || layer:'{layer}'")
            model = Sequential([
    
                    Conv2D(filters=16, kernel_size=(3, 3), activation=activation, input_shape=(100, 100, 1)),
                    MaxPooling2D(pool_size=(2, 2)),

                    Flatten(),
                    Dense(layer, activation=activation),
                    Dense(6, activation="softmax")
                ])
            
            model.compile(optimizer=SGD(lr=0.01), loss="categorical_crossentropy", metrics=["accuracy"])
            model.fit(trainX, trainY, batch_size=64, epochs=20)
            loss, accuracy = model.evaluate(testX, testY)
            print(f"Test Accuracy with activation:'{activation}' || layer:'{layer}' = {accuracy}\n")
            results["Model"].append("CNN-1-CONV")
            results["Input Size"].append("100x100x1")
            results["Activation Func"].append(activation)
            results["Hidden Layer Size"].append(layer)
            results["Learnin Rate"].append(0.01)
            results["Accuracy"].append(accuracy)

Experiment with activation:'relu' || layer:'64'
Epoch 1/20
150/150 [==============================] - 9s 57ms/step - loss: 1.7897 - accuracy: 0.1790
Epoch 2/20
150/150 [==============================] - 8s 57ms/step - loss: 1.7791 - accuracy: 0.2145
Epoch 3/20
150/150 [==============================] - 8s 56ms/step - loss: 1.7656 - accuracy: 0.2300
Epoch 4/20
150/150 [==============================] - 9s 57ms/step - loss: 1.7547 - accuracy: 0.2485
Epoch 5/20
150/150 [==============================] - 9s 57ms/step - loss: 1.7430 - accuracy: 0.2586
Epoch 6/20
150/150 [==============================] - 9s 57ms/step - loss: 1.7285 - accuracy: 0.2705
Epoch 7/20
150/150 [==============================] - 9s 57ms/step - loss: 1.7126 - accuracy: 0.2791
Epoch 8/20
150/150 [==============================] - 9s 59ms/step - loss: 1.6976 - accuracy: 0.2926
Epoch 9/20
150/150 [==============================] - 9s 61ms/step - loss: 1.6851 - accuracy: 0.3023
Epoch 10/20
150/150 [======================

150/150 [==============================] - 10s 68ms/step - loss: 1.7945 - accuracy: 0.1695
Epoch 15/20
150/150 [==============================] - 10s 69ms/step - loss: 1.7947 - accuracy: 0.1661
Epoch 16/20
150/150 [==============================] - 10s 69ms/step - loss: 1.7942 - accuracy: 0.1608
Epoch 17/20
150/150 [==============================] - 10s 68ms/step - loss: 1.7938 - accuracy: 0.1620
Epoch 18/20
150/150 [==============================] - 10s 68ms/step - loss: 1.7949 - accuracy: 0.1631
Epoch 19/20
150/150 [==============================] - 10s 70ms/step - loss: 1.7940 - accuracy: 0.1693
Epoch 20/20
75/75 [==============================] - 1s 10ms/step - loss: 1.7953 - accuracy: 0.1667
Test Accuracy with activation:'sigmoid' || layer:'128' = 0.1666666716337204



# CNN with 2 hidden layers

In [16]:
activations = ['relu', 'sigmoid']
for layer in [64,128]:
    for activation in activations:
            print(f"Experiment with activation:'{activation}' || layer:'{[layer, layer//2]}'")
            model = Sequential([
    
                    Conv2D(filters=16, kernel_size=(3, 3), activation=activation, input_shape=(100, 100, 1)),
                    MaxPooling2D(pool_size=(2, 2)),
                    
                    Conv2D(filters=32, kernel_size=(3, 3), activation=activation),
                    MaxPooling2D(pool_size=(2, 2)),

                    Flatten(),
                    Dense(layer, activation=activation),
                    Dense((layer//2), activation=activation),
                    Dense(6, activation="softmax")
                ])
            
            model.compile(optimizer=SGD(lr=0.01), loss="categorical_crossentropy", metrics=["accuracy"])
            model.fit(trainX, trainY, batch_size=64, epochs=20)
            loss, accuracy = model.evaluate(testX, testY)
            print(f"Test Accuracy with activation:'{activation}' || layer:'{[layer, layer//2]}' = {accuracy}\n")
            results["Model"].append("CNN-2-CONV")
            results["Input Size"].append("100x100x1")
            results["Activation Func"].append(activation)
            results["Hidden Layer Size"].append(layer)
            results["Learnin Rate"].append(0.01)
            results["Accuracy"].append(accuracy)

Experiment with activation:'relu' || layer:'[64, 32]'
Epoch 1/20
150/150 [==============================] - 16s 107ms/step - loss: 1.7908 - accuracy: 0.1807
Epoch 2/20
150/150 [==============================] - 16s 109ms/step - loss: 1.7856 - accuracy: 0.2026
Epoch 3/20
150/150 [==============================] - 16s 107ms/step - loss: 1.7806 - accuracy: 0.2174
Epoch 4/20
150/150 [==============================] - 16s 107ms/step - loss: 1.7722 - accuracy: 0.2258
Epoch 5/20
150/150 [==============================] - 16s 105ms/step - loss: 1.7640 - accuracy: 0.2389
Epoch 6/20
150/150 [==============================] - 16s 107ms/step - loss: 1.7558 - accuracy: 0.2450
Epoch 7/20
150/150 [==============================] - 15s 102ms/step - loss: 1.7462 - accuracy: 0.2596
Epoch 8/20
150/150 [==============================] - 15s 103ms/step - loss: 1.7384 - accuracy: 0.2655
Epoch 9/20
150/150 [==============================] - 16s 107ms/step - loss: 1.7269 - accuracy: 0.2704
Epoch 10/20
150/150

150/150 [==============================] - 18s 117ms/step - loss: 1.7934 - accuracy: 0.1659
Epoch 14/20
150/150 [==============================] - 18s 118ms/step - loss: 1.7932 - accuracy: 0.1668
Epoch 15/20
150/150 [==============================] - 18s 117ms/step - loss: 1.7932 - accuracy: 0.1624
Epoch 16/20
150/150 [==============================] - 18s 117ms/step - loss: 1.7930 - accuracy: 0.1628
Epoch 17/20
150/150 [==============================] - 18s 120ms/step - loss: 1.7931 - accuracy: 0.1669
Epoch 18/20
150/150 [==============================] - 18s 117ms/step - loss: 1.7928 - accuracy: 0.1693
Epoch 19/20
150/150 [==============================] - 18s 118ms/step - loss: 1.7932 - accuracy: 0.1670
Epoch 20/20
75/75 [==============================] - 1s 14ms/step - loss: 1.7928 - accuracy: 0.1667
Test Accuracy with activation:'sigmoid' || layer:'[128, 64]' = 0.1666666716337204



# Evaluation

In [17]:
df = pd.DataFrame.from_dict(results)
df.head(50)

,Model,Input Size,Activation Func,Hidden Layer Size,Learnin Rate,Accuracy
0,MLP,100x100x1,sigmoid,[],0.005,0.157917
1,MLP,100x100x1,sigmoid,[],0.010,0.165000
2,MLP,100x100x1,sigmoid,[],0.017,0.158750
3,MLP,100x100x1,softmax,[],0.005,0.181250
4,MLP,100x100x1,softmax,[],0.010,0.160833
5,MLP,100x100x1,softmax,[],0.017,0.159583
6,MLP,100x100x1,sigmoid,[64],0.005,0.165000
7,MLP,100x100x1,sigmoid,[64],0.010,0.159583
8,MLP,100x100x1,sigmoid,[64],0.017,0.172917
9,MLP,100x100x1,softmax,[64],0.005,0.151250


Here we can see our results. To be able to compare them we will sort them by accuracy.

In [20]:
df_sorted = df.sort_values(by='Accuracy', ascending=False)
df_sorted.head(50)

,Model,Input Size,Activation Func,Hidden Layer Size,Learnin Rate,Accuracy
32,CNN-1-CONV,100x100x1,relu,128,0.010,0.333333
36,CNN-2-CONV,100x100x1,relu,128,0.010,0.332917
34,CNN-2-CONV,100x100x1,relu,64,0.010,0.330417
30,CNN-1-CONV,100x100x1,relu,64,0.010,0.327500
19,MLP,100x100x1,sigmoid,"[128, 64]",0.010,0.185417
21,MLP,100x100x1,softmax,"[128, 64]",0.005,0.185000
23,MLP,100x100x1,softmax,"[128, 64]",0.017,0.182500
13,MLP,100x100x1,sigmoid,[128],0.010,0.182083
3,MLP,100x100x1,softmax,[],0.005,0.181250
15,MLP,100x100x1,softmax,[128],0.005,0.179583


If we look at the table above we can see the performance of various neural networks with different hyperparameters like activation function, hidden layer size and leraning rate. What interesting is that the CNN models do not consistently outperform MLP models in this context. Model 19, an MLP with sigmoid activation and [128, 64] hidden layers, stands out with a little bit higher accuracy. However, trying different activation functions, different hidden layer sizes and different learning rates didnt change the results too much. For some reason both MLP and CNN could not be succesfully trained with the given dataset. I tried many different approaches like using RGB input, changing input sizes, building more complex neural networks but none of them worked. There is only one exception as can be seen from the table which is the CNN with relu activation functions. Among these models a CNN with 1 convolutional layer and [128] hidden layer was the most successful one with an accuracy of 0.333333. But this is still far away from a good optimal model. Despite many efforts and attempts, I still could not achieve good results and unfortunately I still do not know what the reason for this is.